# Titouan Mendiharat's report

In [2]:
# Imports
import numpy as np
import scipy.sparse as sp
from scipy.linalg import norm, solve
from scipy.optimize import check_grad
import matplotlib.pyplot as plt
import time
from numpy.linalg import norm

In [3]:
def load_data(file_name_matrix='tfidf_matrix_97MB.npz', file_name_feature_names='feature_names_97MB.npy',
	      file_name_labels='train_labels.npy', samples_in_train_set=10000,
	      samples_in_test_set=137562):
	# Recuperation des donnees
	TF_IDF_matrix = sp.load_npz(file_name_matrix)
	TF_IDF_feature_names = np.load(file_name_feature_names)
	train_labels = np.load(file_name_labels, allow_pickle=True)
	train_labels_numeric = ((train_labels == 'Oui') + 0)

	X = TF_IDF_matrix[:samples_in_train_set].toarray()
	y = train_labels_numeric[:samples_in_train_set] * 2 - 1

	X_test = TF_IDF_matrix[samples_in_train_set:samples_in_train_set+samples_in_test_set].toarray()
	y_test = train_labels_numeric[samples_in_train_set:samples_in_train_set+samples_in_test_set] * 2 - 1


	# Standardisation des données
	std_X = np.maximum(np.std(X, axis=0), 1e-7)
	X = X / std_X
	X_test = X_test / std_X

	n = X.shape[0]
	n_test = X_test.shape[0]
	m = X.shape[1]

	# Ajout d'une colonne de uns
	eX = np.hstack((np.ones((n,1)), X))
	eX_test = np.hstack((np.ones((n_test,1)), X_test))

	return eX, y, eX_test, y_test

## Question 3.1

![alternative text](f1grad.jpg)

![alternative text](f1hess.jpg)

Hence, the function $f_1$ is convex.

## Question 3.2

In [4]:
# First we need to load the data

eX, y, eX_test, y_test = load_data()
eX_transpose = eX.T
n = eX.shape[0]


In [5]:
def f1(w0,w) :
    sum = 0
    for i in range(n) :
        exp = np.exp(-y[i]*(eX_transpose[i]*w+w0))
        sum += np.log10(1+exp)
    sum = sum/n + (norm(w)**2)/(2*n)
    return sum

def gradf1(w0,w) :
    sum1 = 0
    sum2 = 0
    for i in range(n) :
        exp = np.exp(-y[i]*(eX_transpose[i]*w+w0))
        sum1 += (-y[i]*exp)/(1+exp)
        sum2 += (-y[i]*eX[i]*exp)/(1+exp)
    sum1 = sum1/n
    sum2 = sum2/n + norm(w)/n
    return [sum1, sum2]

def hessf1(w0,w) :
    sum1 = 0
    sum2 = 0
    sum3 = 0
    for i in range(n) :
        exp = np.exp(-y[i]*(eX_transpose[i]*w+w0))
        sum1 += ((y[i]**2)*exp)/((1+exp)**2)
        sum2 += ((y[i]**2)*eX_transpose[i]*exp)/((1+exp)**2)
        sum3 += ((y[i]**2)*eX[i]*eX_transpose[i]*exp)/((1+exp)**2)
    sum1 = sum1/n
    sum2 = sum2/n
    sum3 = sum3/n + 1/n
    return [[sum1, sum2], [sum2, sum3]]


## Question 3.3

In [ ]:
def newton(w0, w, max_iter=100): 
    iteration = []
    log_grad = []
    for i in range(max_iter):
        grad = gradf1(w0, w)
        hess = hessf1(w0, w)
        h = np.linalg.solve(hess, grad)
        w0 = w0 - h[0]
        w = w - h[1]
        iteration.append(i)
        log_grad.append(np.log(grad))
        if np.linalg.norm(h) < 10**(-10):
            break
    plt.plot(iteration, log_grad)
    return [w0, w]

newton(0,0)

## Question 3.4

In [ ]:
newton(1,1)

## Question 3.5

## Question 4.1

La méthode de newton ne peux pas ête utilisée car la matrice Hessienne de f1 n'est pas inversible.
On ne peut donc pas résoudre le système d'équations et trouver h dans mon code.

## Question 4.2